Creación de un dashboard interactivo para visualizar el impacto de una campaña de marketing para un banco. Técnicas: Conexión a bases de datos en Google Cloud platform, modelado de datos, creación de visualizaciones interactivas y publicación en Looker Studio y Tableau. Dataset: Banking Dataset - Marketing Targets (Kaggle)

# Configuraciones iniciales y preparación del dataset y entorno de Google Cloud Platform

In [2]:
# -------------------------------
# 1️⃣ Importar librerías
# -------------------------------
import pandas as pd
from google.cloud import bigquery

In [3]:
# -------------------------------
# 2️⃣ Configurar cliente BigQuery
# -------------------------------
# Descarga tu archivo de credenciales JSON de GCP y subelo a Colab
# Reemplaza 'tu_clave.json' con el nombre de tu archivo
client = bigquery.Client.from_service_account_json('etldatascience-1dd3397f3376.json')

In [4]:
# -------------------------------
# 3️⃣ Leer dataset CSV de Kaggle
# -------------------------------
# Train dataset
df_train = pd.read_csv('train.csv', sep=';')

# Revisar primeras filas
print(df_train.head())
print(df_train.info())

   age           job  marital  education default  balance housing loan  \
0   58    management  married   tertiary      no     2143     yes   no   
1   44    technician   single  secondary      no       29     yes   no   
2   33  entrepreneur  married  secondary      no        2     yes  yes   
3   47   blue-collar  married    unknown      no     1506     yes   no   
4   33       unknown   single    unknown      no        1      no   no   

   contact  day month  duration  campaign  pdays  previous poutcome   y  
0  unknown    5   may       261         1     -1         0  unknown  no  
1  unknown    5   may       151         1     -1         0  unknown  no  
2  unknown    5   may        76         1     -1         0  unknown  no  
3  unknown    5   may        92         1     -1         0  unknown  no  
4  unknown    5   may       198         1     -1         0  unknown  no  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45211 entries, 0 to 45210
Data columns (total 17 columns):
 #

In [5]:
# -------------------------------
# 4️⃣ Crear columnas adicionales para análisis
# -------------------------------
# Objetivo binario
df_train['y_binary'] = df_train['y'].map({'yes': 1, 'no': 0})

# Duración en minutos
df_train['duration_min'] = df_train['duration'] / 60

In [6]:
# -------------------------------
# 5️⃣ Guardar CSV limpio
# -------------------------------
df_train.to_csv('train_clean.csv', index=False)
print("CSV limpio listo para BigQuery")

CSV limpio listo para BigQuery


In [7]:
# -------------------------------
# 6️⃣ Subir DataFrame a BigQuery
# -------------------------------
# Configurar proyecto y dataset en GCP
project_id = 'etldatascience'
dataset_id = 'bank_marketing'
table_id = f'{project_id}.{dataset_id}.marketing_targets'

for col in ["default", "housing", "loan", "y"]:
    df_train[col] = df_train[col].map({"yes": True, "no": False})

# Subir datos
job = client.load_table_from_dataframe(df_train, table_id)
job.result()  # Espera a que termine la carga
print("Datos cargados a BigQuery correctamente")

Datos cargados a BigQuery correctamente


In [8]:
# -------------------------------
# 7️⃣ Consultas SQL
# -------------------------------
# Total de contactos
query_total = f"""
SELECT COUNT(*) AS total_contacts
FROM `{table_id}`
"""
total_contacts = client.query(query_total).to_dataframe()
print("Total de contactos:\n", total_contacts)

# Tasa de conversión
query_conversion = f"""
SELECT SUM(y_binary)/COUNT(*) AS conversion_rate
FROM `{table_id}`
"""
conversion_rate = client.query(query_conversion).to_dataframe()
print("Tasa de conversión:\n", conversion_rate)

# Duración promedio por campaña
query_duration = f"""
SELECT campaign, AVG(duration_min) AS avg_duration
FROM `{table_id}`
GROUP BY campaign
ORDER BY avg_duration DESC
"""
avg_duration = client.query(query_duration).to_dataframe()
print("Promedio duración por campaña:\n", avg_duration.head())


Total de contactos:
    total_contacts
0          180844
Tasa de conversión:
    conversion_rate
0         0.116985
Promedio duración por campaña:
    campaign  avg_duration
0        55     18.233333
1         2      4.588767
2         3      4.527314
3         1      4.350894
4         4      4.219908


# Modelo predictivo con Machine Learning

In [9]:
# =========================================
# 1) Imports e instalación opcional
# =========================================

import numpy as np
import pandas as pd
from google.cloud import bigquery
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (roc_auc_score, average_precision_score, f1_score,
                             precision_recall_curve, classification_report,
                             confusion_matrix)

In [10]:
# =========================================
# 2) Conexión a BigQuery
# =========================================
# Datos previamente usados:
SERVICE_ACCOUNT_JSON = "etldatascience-1dd3397f3376.json"
PROJECT_ID = "etldatascience"
DATASET = "bank_marketing"
TABLE = "marketing_targets"
TABLE_ID = f"{PROJECT_ID}.{DATASET}.{TABLE}"

client = bigquery.Client.from_service_account_json(SERVICE_ACCOUNT_JSON)

In [11]:
# =========================================
# 3) Cargar datos desde BigQuery
#    (usamos todas las columnas y luego elegimos features)
# =========================================
query = f"""
SELECT *
FROM `{TABLE_ID}`
"""
df = client.query(query).to_dataframe()

print("Shape:", df.shape)
print(df.head(3))
print(df.dtypes)

Shape: (180844, 19)
   age           job  marital  education  default  balance  housing   loan  \
0   58    management  married   tertiary    False     2143     True  False   
1   44    technician   single  secondary    False       29     True  False   
2   33  entrepreneur  married  secondary    False        2     True   True   

   contact  day month  duration  campaign  pdays  previous poutcome      y  \
0  unknown    5   may       261         1     -1         0  unknown  False   
1  unknown    5   may       151         1     -1         0  unknown  False   
2  unknown    5   may        76         1     -1         0  unknown  False   

   y_binary  duration_min  
0         0      4.350000  
1         0      2.516667  
2         0      1.266667  
age               Int64
job              object
marital          object
education        object
default         boolean
balance           Int64
housing         boolean
loan            boolean
contact          object
day               Int64
mo

In [12]:
# =========================================
# 4) Definir target y escenarios de features
#    Escenario recomendado: PRE-CALL (sin 'duration' ni 'duration_min')
# =========================================
TARGET = "y_binary"  # 1 si convierte, 0 si no
USE_DURATION = False # cambia a True si quieres el escenario post-call

# Categóricas del dataset original
cat_cols = ["job","marital","education","default","housing","loan",
            "contact","month","poutcome"]
# Numéricas del dataset original (sin leakage)
num_cols = ["age","balance","day","campaign","pdays","previous"]

# Si quieres escenario post-call (incluye duración conocida tras la llamada)
if USE_DURATION:
    # Puedes usar 'duration_min' (tu columna calculada); evita mezclar ambas
    num_cols = num_cols + ["duration_min"]

# Drop de filas con target nulo (por si acaso)
df = df.dropna(subset=[TARGET]).copy()

X = df[cat_cols + num_cols]
y = df[TARGET].astype(int)

# Checar balance de clases
print("Distribución de y:", y.value_counts(normalize=True).round(4))

Distribución de y: y_binary
0    0.883
1    0.117
Name: proportion, dtype: float64


In [13]:
# =========================================
# 5) Preprocesamiento: imputación + OHE + escalado
# =========================================
numeric_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore"))
])

preprocess = ColumnTransformer(
    transformers=[
        ("num", numeric_transformer, num_cols),
        ("cat", categorical_transformer, cat_cols),
    ]
)

In [14]:
# =========================================
# 6) Train/Test split estratificado
# =========================================
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Reemplazar cualquier pd.NA por np.nan de forma segura
X_train = X_train.applymap(lambda x: np.nan if pd.isna(x) else x)
X_test = X_test.applymap(lambda x: np.nan if pd.isna(x) else x)

/tmp/ipython-input-2786071077.py:9: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  X_train = X_train.applymap(lambda x: np.nan if pd.isna(x) else x)
/tmp/ipython-input-2786071077.py:10: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  X_test = X_test.applymap(lambda x: np.nan if pd.isna(x) else x)


In [15]:
# =========================================
# 7) Modelos base
#    A) Regresión Logística
# =========================================
log_reg = Pipeline(steps=[
    ("prep", preprocess),
    ("clf", LogisticRegression(max_iter=2000, class_weight="balanced"))
])

# Validación cruzada con varias métricas
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = {"roc_auc":"roc_auc", "pr_auc":"average_precision", "f1":"f1"}

cv_log = cross_validate(log_reg, X_train, y_train, cv=cv, scoring=scoring, n_jobs=-1, return_train_score=False)
print("\n[LogReg] CV ROC-AUC: ", np.mean(cv_log["test_roc_auc"]).round(4))
print("[LogReg] CV PR-AUC : ", np.mean(cv_log["test_pr_auc"]).round(4))
print("[LogReg] CV F1     : ", np.mean(cv_log["test_f1"]).round(4))

# Fit y evaluación en test con threshold por defecto (0.5)
log_reg.fit(X_train, y_train)
proba_lr = log_reg.predict_proba(X_test)[:,1]
print("\n[LogReg] Test ROC-AUC:", roc_auc_score(y_test, proba_lr).round(4))
print("[LogReg] Test PR-AUC :", average_precision_score(y_test, proba_lr).round(4))

# Tuning de umbral para maximizar F1 (puedes cambiar por recall/precision balanceados)
prec, rec, thr = precision_recall_curve(y_test, proba_lr)
f1s = (2*prec*rec)/(prec+rec+1e-9)
best_idx = np.argmax(f1s[:-1])  # thr tiene len = len(prec)-1
best_thr = thr[best_idx]
print("[LogReg] Mejor umbral F1:", best_thr.round(3), " | F1:", f1s[best_idx].round(4))

y_pred_lr = (proba_lr >= best_thr).astype(int)
print("\n[LogReg] Classification report (umbral optimizado)")
print(classification_report(y_test, y_pred_lr, digits=4))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred_lr))


[LogReg] CV ROC-AUC:  0.7659
[LogReg] CV PR-AUC :  0.3983
[LogReg] CV F1     :  0.3757

[LogReg] Test ROC-AUC: 0.7673
[LogReg] Test PR-AUC : 0.4041
[LogReg] Mejor umbral F1: 0.649  | F1: 0.4434

[LogReg] Classification report (umbral optimizado)
              precision    recall  f1-score   support

           0     0.9251    0.9323    0.9287     31938
           1     0.4572    0.4304    0.4434      4231

    accuracy                         0.8736     36169
   macro avg     0.6912    0.6814    0.6860     36169
weighted avg     0.8704    0.8736    0.8719     36169

Confusion matrix:
 [[29776  2162]
 [ 2410  1821]]


In [16]:
# =========================================
# 7B) Random Forest (robusto con categóricas OHE)
# =========================================
rf = Pipeline(steps=[
    ("prep", preprocess),
    ("clf", RandomForestClassifier(
        n_estimators=400,
        max_depth=None,
        min_samples_split=10,
        min_samples_leaf=3,
        class_weight="balanced",
        n_jobs=-1,
        random_state=42
    ))
])

cv_rf = cross_validate(rf, X_train, y_train, cv=cv, scoring=scoring, n_jobs=-1, return_train_score=False)
print("\n[RF] CV ROC-AUC: ", np.mean(cv_rf["test_roc_auc"]).round(4))
print("[RF] CV PR-AUC : ", np.mean(cv_rf["test_pr_auc"]).round(4))
print("[RF] CV F1     : ", np.mean(cv_rf["test_f1"]).round(4))

rf.fit(X_train, y_train)
proba_rf = rf.predict_proba(X_test)[:,1]
print("\n[RF] Test ROC-AUC:", roc_auc_score(y_test, proba_rf).round(4))
print("[RF] Test PR-AUC :", average_precision_score(y_test, proba_rf).round(4))

prec, rec, thr = precision_recall_curve(y_test, proba_rf)
f1s = (2*prec*rec)/(prec+rec+1e-9)
best_idx = np.argmax(f1s[:-1])
best_thr_rf = thr[best_idx]
print("[RF] Mejor umbral F1:", best_thr_rf.round(3), " | F1:", f1s[best_idx].round(4))

y_pred_rf = (proba_rf >= best_thr_rf).astype(int)
print("\n[RF] Classification report (umbral optimizado)")
print(classification_report(y_test, y_pred_rf, digits=4))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred_rf))


[RF] CV ROC-AUC:  0.9649
[RF] CV PR-AUC :  0.8393
[RF] CV F1     :  0.7658

[RF] Test ROC-AUC: 0.98
[RF] Test PR-AUC : 0.8977
[RF] Mejor umbral F1: 0.57  | F1: 0.8295

[RF] Classification report (umbral optimizado)
              precision    recall  f1-score   support

           0     0.9773    0.9776    0.9774     31938
           1     0.8306    0.8284    0.8295      4231

    accuracy                         0.9602     36169
   macro avg     0.9039    0.9030    0.9035     36169
weighted avg     0.9601    0.9602    0.9601     36169

Confusion matrix:
 [[31223   715]
 [  726  3505]]


In [17]:
# =========================================
# 8) Importancia de características
#    A) Coeficientes de la Regresión Logística
# =========================================
# Obtenemos nombres de columnas transformadas:
ohe = log_reg.named_steps["prep"].named_transformers_["cat"].named_steps["onehot"]
cat_feature_names = ohe.get_feature_names_out(cat_cols)
feature_names = np.r_[num_cols, cat_feature_names]

coefs = log_reg.named_steps["clf"].coef_.ravel()
coef_df = pd.DataFrame({"feature": feature_names, "coef": coefs})
coef_df["abs_coef"] = coef_df["coef"].abs()
coef_top = coef_df.sort_values("abs_coef", ascending=False).head(20)
print("\nTop 20 features (LogReg | |coef|):\n", coef_top[["feature","coef"]])


Top 20 features (LogReg | |coef|):
               feature      coef
48   poutcome_success  1.711139
41          month_mar  1.337901
38          month_jan -1.026119
44          month_oct  0.985393
45          month_sep  0.857613
43          month_nov -0.792029
36          month_dec  0.749604
35          month_aug -0.712786
33    contact_unknown -0.710739
39          month_jul -0.651656
46   poutcome_failure -0.635720
49   poutcome_unknown -0.600174
31   contact_cellular  0.535785
11        job_retired  0.498836
42          month_may -0.486496
14        job_student  0.429926
47     poutcome_other -0.362090
37          month_feb -0.329891
9       job_housemaid -0.297021
32  contact_telephone  0.288109


In [28]:
# =========================================
# 9) Guardar artefactos (opcional)
# =========================================
import joblib, os
os.makedirs("artifacts", exist_ok=True)
joblib.dump(log_reg, "artifacts/model_logreg_pre.pkl")
joblib.dump(rf,      "artifacts/model_rf_pre.pkl")
coef_top.to_csv("artifacts/top_features_logreg.csv", index=False)
print("\nArtefactos guardados en carpeta 'artifacts/'.")


Artefactos guardados en carpeta 'artifacts/'.


**Conclusión:**

El análisis de la campaña de marketing muestra que los factores más determinantes para la conversión de clientes son los resultados de campañas previas exitosas (poutcome_success), el canal de contacto empleado (contact_cellular) y la estacionalidad reflejada en ciertos meses del año. Mientras que algunas variables demográficas como el tipo de empleo (job_student, job_retired) aportan información adicional, su impacto es menor. Estos hallazgos permiten identificar estrategias de segmentación más efectivas y priorizar recursos en clientes con mayor probabilidad de conversión, facilitando la toma de decisiones basada en datos para futuras campañas.